In [1]:
#basic
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from sklearn.decomposition import PCA
#Pretreatment
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,Normalizer
from sklearn.feature_selection import VarianceThreshold
#model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,NuSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
cols = [c for c in train.columns if c not in ['id', 'target','wheezy-copper-turtle-magic']]
oof_qda = np.zeros(len(train)) 
pred_qda = np.zeros(len(test))

for i in tqdm_notebook(range(512)):
    train1 = train[train['wheezy-copper-turtle-magic']==i]
    test1 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train1.index; idx2 = test1.index
    train1.reset_index(drop=True,inplace=True)
    
    data = pd.concat([pd.DataFrame(train1[cols]), pd.DataFrame(test1[cols])])
    data2 =VarianceThreshold(threshold=1.5).fit_transform(data[cols])
    train2 = data2[:train1.shape[0]]; test2 = data2[train1.shape[0]:]
    
    
    #oof
    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train1, train1['target']):

        clf = QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof_qda[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        pred_qda[idx2] += clf.predict_proba(test2)[:,1] / skf.n_splits
                
       
        
print('QDA scores CV', roc_auc_score(train['target'], oof_qda))


QDA scores CV 0.9648973416075834


In [4]:
# test예측값 할당
test['target'] = pred_qda

oof_qda_pl = np.zeros(len(train)) 
pred_qda_pl = np.zeros(len(test))

for i in tqdm_notebook(range(512)):
    train1 = train[train['wheezy-copper-turtle-magic']==i]
    train_pl = train1.copy()
    test1 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train1.index; idx2 = test1.index
    
    #pseudo labeling
    test_pl = test1[(test1['target']>0.99) | (test1['target']<0.01)].copy()
    test_pl.loc[ test_pl['target']>=0.5, 'target' ] = 1
    test_pl.loc[ test_pl['target']< 0.5, 'target' ] = 0 
    train_pl2 =  pd.concat([train_pl,test_pl],axis=0)
    train_pl2.reset_index(drop=True,inplace=True)
    
    useful = VarianceThreshold(threshold=1.5).fit(train_pl2[cols])     
    train_pl3 = useful.transform(train_pl2[cols])
    train2 = useful.transform(train1[cols])
    test2 = useful.transform(test1[cols])
    
    #oof
    skf = StratifiedKFold(n_splits=11, random_state=42,shuffle=True)
    for train_index, test_index in skf.split(train_pl3, train_pl2['target']):
        
        #pseudo label을 무시하는 코드 중요!!
        # 처음 StratifiedKFold로 분할할 때 shuffle=True옵션을 안넣어줘 계속 오류발생
        test_index3 = test_index[ test_index<len(train2) ]
        
        clf = QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train_pl3[train_index,:],train_pl2.loc[train_index]['target'])
        oof_qda_pl[idx1[test_index3]] = clf.predict_proba(train2[test_index3,:])[:,1]
        pred_qda_pl[idx2] += clf.predict_proba(test2)[:,1] / skf.n_splits
                
       
     
print('Pseudo Labeled QDA scores CV =',round(roc_auc_score(train['target'],oof_qda_pl),5))
    


Pseudo Labeled QDA scores CV = 0.97024
